##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Herramienta de autoría TFLite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/guide/authoring"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

La API de autoría de TensorFlow Lite proporciona una forma de mantener sus modelos `tf.function` compatibles con TensorFlow Lite.


## Configurar

In [ ]:
import tensorflow as tf

## Problema de compatibilidad entre TensorFlow y TensorFlow Lite

Si desea usar su modelo TF en dispositivos, necesita convertirlo a un modelo TFLite para poder usarlo desde el intérprete TFLite. Durante la conversión, podría encontrarse con un error de compatibilidad debido a ops de TensorFlow no admitidas por el conjunto de ops incorporadas de TFLite.

Se trata de una cuestión un tanto molesta. ¿Cómo se puede detectar antes, como en el momento de la creación del modelo?

Tenga en cuenta que el siguiente código fallará en la llamada `converter.convert()`.


In [ ]:
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")

In [ ]:
# Convert the tf.function
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [f.get_concrete_function()], f)
try:
  fb_model = converter.convert()
except Exception as e:
  print(f"Got an exception: {e}")

## Uso sencillo de la autoría consciente del objetivo

Hemos implantado la API de autoría para detectar el problema de compatibilidad con TensorFlow Lite en el momento de la creación del modelo.

Sólo tiene que añadir el decorador `@tf.lite.experimental.authoring.compatible` para encapsular su modelo `tf.function` para comprobar la compatibilidad con TFLite.

Después de esto, la compatibilidad se comprobará automáticamente cuando evalúe su modelo.

In [ ]:
@tf.lite.experimental.authoring.compatible
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


Si se encuentra algún problema de compatibilidad con TensorFlow Lite, aparecerá `COMPATIBILITY WARNING` o `COMPATIBILITY ERROR` con la localización exacta de la op problemática. En este ejemplo, muestra la ubicación de `tf.Cosh` op en su modelo tf.function.

También puede verificar el registro de compatibilidad con el método `<function_name>.get_compatibility_log()`.

In [ ]:
compatibility_log = '\n'.join(f.get_compatibility_log())
print (f"compatibility_log = {compatibility_log}")

## Lanzar una excepción por incompatibilidad

Puede dar una opción al decorador `@tf.lite.experimental.authoring.compatible`. La opción `raise_exception` emite una excepción cuando se intenta evaluar el modelo decorado.

In [ ]:
@tf.lite.experimental.authoring.compatible(raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
try:
  result = f(tf.constant([0.0]))
  print (f"result = {result}")
except Exception as e:
  print(f"Got an exception: {e}")

## Especificación del uso de "Select TF ops"

Si ya conoce el uso de [Select TF ops](https://www.tensorflow.org/lite/guide/ops_select), puede indicárselo a la API de creación configurando `converter_target_spec`. Es el mismo objeto [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) que usará para la API [tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter).


In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec, raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


## Comprobar la compatibilidad de la GPU

Si quiere asegurarse de que su modelo es compatible con el [delegado GPU](https://www.tensorflow.org/lite/performance/gpu) de TensorFlow Lite, puede configurar `experimental_supported_backends` de [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec).

El siguiente ejemplo muestra cómo garantizar la compatibilidad de su modelo con los delegados de la GPU. Tenga en cuenta que este modelo tiene problemas de compatibilidad ya que usa un tensor 2D con el operador tf.slice y el operador tf.cosh no soportado. Verá dos `COMPATIBILITY WARNING` con la información de localización.

In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
target_spec.experimental_supported_backends = ["GPU"]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  y = tf.cosh(x)
  return y + tf.slice(x, [1, 1], [1, 1])

result = func(tf.ones(shape=(4,4), dtype=tf.float32))

## Más información

Para más información, consulte:

- Documento de la API [tf.function](https://www.tensorflow.org/api_docs/python/tf/function)
- [Mejor rendimiento con tf.function](https://www.tensorflow.org/guide/function)
- [Convertidor de TensorFlow Lite](https://www.tensorflow.org/lite/models/convert)
- [Analizador de modelo de TensorFlow](https://www.tensorflow.org/lite/guide/model_analyzer)
- [Delegado de GPU de TensorFlow Lite](https://www.tensorflow.org/lite/performance/gpu)